In [10]:
from pinecone import Pinecone, ServerlessSpec

pinecone_key = "pcsk_4PR9Ek_TvzdWZi5dsDCvKCZStfatGWfV5Mog5hbS6ccmZGd4W5bA6Q5EAygQ1LQRVVxn83"

pc = Pinecone(api_key=pinecone_key)

index_name = "first-index"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )
else:
    print(f"Index `{index_name}` already exists")

Index `first-index` already exists


In [11]:
index = pc.Index(index_name)

c:\Users\alikh\anaconda3\envs\langchain-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'langchain': {'vector_count': 10}, 'ns1': {'vector_count': 10}},
 'total_vector_count': 20,
 'vector_type': 'dense'}

In [ ]:
import uuid

parsed_docs_with_ids = [
    {"_id": str(uuid.uuid4()), **{("text" if k == "TITLE" else k): v for k, v in doc.items()}}
    for doc in parsed_docs
]

index.upsert_records("langchain", parsed_docs_with_ids)

In [21]:
from helpers import parse_snippet
from helpers import pretty_print_dict
import uuid

my_lib_docs = """
TITLE: Calculate the Mean of a List  
DESCRIPTION: Computes the average (arithmetic mean) of a list of numbers using the `calculate_mean()` function in `project_demo`. Handles both integers and floats.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/stats.py#L10  
LANGUAGE: python  
CODE:
```
from project_demo import calculate_mean

nums = [1, 2, 3, 4, 5]
mean = calculate_mean(nums)
print(mean)  # 3.0
```

-------------------------------------

TITLE: Normalize a List of Numbers  
DESCRIPTION: Demonstrates how to normalize a list of numbers between 0 and 1 using the `normalize()` function from `project_demo`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/stats.py#L25  
LANGUAGE: python  
CODE:
```
from project_demo import normalize

data = [10, 20, 30, 40]
norm = normalize(data)
print(norm)  # [0.0, 0.333, 0.666, 1.0]
```

-------------------------------------

TITLE: Count Vowels in a String  
DESCRIPTION: Counts the number of vowels in a string using the `count_vowels()` function in `project_demo.string_utils`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/string_utils.py#L12  
LANGUAGE: python  
CODE:
```
from project_demo import count_vowels

s = "OpenAI ChatGPT"
print(count_vowels(s))  # 5
```

-------------------------------------

TITLE: Reverse Words in a Sentence  
DESCRIPTION: Reverses the order of words in a sentence using `reverse_words()` from `project_demo.string_utils`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/string_utils.py#L27  
LANGUAGE: python  
CODE:
```
from project_demo import reverse_words

sentence = "Hello world from AI"
print(reverse_words(sentence))  # "AI from world Hello"
```

-------------------------------------

TITLE: Generate Fibonacci Sequence  
DESCRIPTION: Generates the first N numbers in the Fibonacci sequence using `fibonacci(n)` from `project_demo.math_utils`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/math_utils.py#L15  
LANGUAGE: python  
CODE:
```
from project_demo import fibonacci

print(fibonacci(6))  # [0, 1, 1, 2, 3, 5]
```

-------------------------------------

TITLE: Check for Palindrome  
DESCRIPTION: Verifies whether a given string is a palindrome using `is_palindrome()` in `project_demo.string_utils`. Ignores case and whitespace.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/string_utils.py#L40  
LANGUAGE: python  
CODE:
```
from project_demo import is_palindrome

print(is_palindrome("Racecar"))      # True
print(is_palindrome("Hello world"))  # False
```

-------------------------------------

TITLE: Convert List to Dictionary  
DESCRIPTION: Converts two lists (keys and values) into a dictionary using `list_to_dict()` in `project_demo.utils`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/utils.py#L5  
LANGUAGE: python  
CODE:
```
from project_demo import list_to_dict

keys = ["a", "b", "c"]
vals = [1, 2, 3]
print(list_to_dict(keys, vals))  # {'a': 1, 'b': 2, 'c': 3}
```

-------------------------------------

TITLE: Flatten a Nested List  
DESCRIPTION: Flattens a 2D list into a 1D list using `flatten()` from `project_demo.utils`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/utils.py#L18  
LANGUAGE: python  
CODE:
```
from project_demo import flatten

nested = [[1, 2], [3, 4], [5]]
print(flatten(nested))  # [1, 2, 3, 4, 5]
```

-------------------------------------

TITLE: Merge Two Sorted Lists  
DESCRIPTION: Merges two sorted lists into a single sorted list using `merge_sorted_lists()` from `project_demo.sorting`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/sorting.py#L9  
LANGUAGE: python  
CODE:
```
from project_demo import merge_sorted_lists

a = [1, 3, 5]
b = [2, 4, 6]
print(merge_sorted_lists(a, b))  # [1, 2, 3, 4, 5, 6]
```

-------------------------------------

TITLE: Calculate Dot Product of Two Vectors  
DESCRIPTION: Computes the dot product of two numeric lists using `dot_product()` from `project_demo.math_utils`.  
SOURCE: https://github.com/project_demo/project_demo/blob/main/math_utils.py#L33  
LANGUAGE: python  
CODE:
```
from project_demo import dot_product

a = [1, 2, 3]
b = [4, 5, 6]
print(dot_product(a, b))  # 32
```
"""

snips = my_lib_docs.split("-------------------------------------")
snips = [snip.strip() for snip in snips]
snips = [parse_snippet(snip) for snip in snips]

snips = [
    {"_id": str(uuid.uuid4()), **{("text" if k == "DESCRIPTION" else k): v for k, v in doc.items()}}
    for doc in snips
]

index.upsert_records("project_demo", snips)

In [18]:
snips[0]

{'_id': '7114c64f-d385-4b9c-96fa-c6cc3217f9f9',
 'text': 'Calculate the Mean of a List',
 'DESCRIPTION': 'Computes the average (arithmetic mean) of a list of numbers using the `calculate_mean()` function in `project_demo`. Handles both integers and floats.',
 'SOURCE': 'https://github.com/project_demo/project_demo/blob/main/stats.py#L10',
 'LANGUAGE': 'python',
 'CODE': 'from project_demo import calculate_mean\n\nnums = [1, 2, 3, 4, 5]\nmean = calculate_mean(nums)\nprint(mean)  # 3.0'}

In [32]:
query = "Conversational bot"

results = index.search(
    namespace="project_demo",
    query={
        "top_k": 3,
        "inputs": {
            'text': query
        }
    }
)

key_words = ['TITLE', 'text', 'LANGUAGE', 'SOURCE', 'CODE']

context = ""

for res in results['result']['hits']:
    fields = res['fields']
    temp = ""
    temp += key_words[0] + ': ' +fields[key_words[0]] + '\n'
    temp += "DESCRIPTION" + ': ' +fields[key_words[1]] + '\n'
    temp += key_words[2] + ': ' +fields[key_words[2]] + '\n'
    temp += key_words[3] + ': ' +fields[key_words[3]] + '\n'
    temp += key_words[4] + ': ' + f"```\n{fields[key_words[4]]}\n```" + '\n'

    context += temp + '----------------------\n'

print(context)

# print(results['result']['hits'])

TITLE: Reverse Words in a Sentence
DESCRIPTION: Reverses the order of words in a sentence using `reverse_words()` from `project_demo.string_utils`.
LANGUAGE: python
SOURCE: https://github.com/project_demo/project_demo/blob/main/string_utils.py#L27
CODE: ```
from project_demo import reverse_words

sentence = "Hello world from AI"
print(reverse_words(sentence))  # "AI from world Hello"
```
----------------------
TITLE: Check for Palindrome
DESCRIPTION: Verifies whether a given string is a palindrome using `is_palindrome()` in `project_demo.string_utils`. Ignores case and whitespace.
LANGUAGE: python
SOURCE: https://github.com/project_demo/project_demo/blob/main/string_utils.py#L40
CODE: ```
from project_demo import is_palindrome

print(is_palindrome("Racecar"))      # True
print(is_palindrome("Hello world"))  # False
```
----------------------
TITLE: Generate Fibonacci Sequence
DESCRIPTION: Generates the first N numbers in the Fibonacci sequence using `fibonacci(n)` from `project_demo.ma

In [18]:
reranked_results = index.search(
    namespace="ns1",
    query={
        "top_k": 5,
        "inputs": {
            'text': query
        }
    },
    rerank={
        "model": "bge-reranker-v2-m3",
        "top_n": 5,
        "rank_fields": ["text"]
    },
    fields=["category", "text"]
)

reranked_results

{'result': {'hits': [{'_id': 'rec1',
                      '_score': 0.10650458931922913,
                      'fields': {'category': 'history',
                                 'text': 'The Eiffel Tower was completed in '
                                         '1889 and stands in Paris, France.'}},
                     {'_id': 'rec7',
                      '_score': 0.06278920918703079,
                      'fields': {'category': 'history',
                                 'text': 'The Great Wall of China was built to '
                                         'protect against invasions.'}},
                     {'_id': 'rec5',
                      '_score': 3.21923362207599e-05,
                      'fields': {'category': 'literature',
                                 'text': 'Shakespeare wrote many famous plays, '
                                         'including Hamlet and Macbeth.'}},
                     {'_id': 'rec4',
                      '_score': 1.6187581422855146e-